<a href="https://colab.research.google.com/github/lisphilar/covid19-sir/blob/master/example/05_scenario_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scenario analysis
We will perform scenario analysis. Regarding phase-dependent SIR-derived ODE models, this analysis focused on the impact of changes of ODE parameter values on the number of cases.

In [ ]:
from datetime import timedelta
import pandas as pd
try:
    import covsirphy as cs
except ImportError:
    !pip install --upgrade "git+https://github.com/lisphilar/covid19-sir.git#egg=covsirphy" -qq
    import covsirphy as cs
cs.__version__

## 1. Create analyzer
An instance of `ODEScenario()` will be created here. We have two options as follows.

- With dataset of recommended servers
- With `pandas.DataFrame`

### 1.1 With dataset of recommended servers
We can create an instance with class method `ODEScenario.auto_build()` if the target dataset was registered in the recommended servers explained in [Data preparation]. As an example, we will use the dataset in Italy.

The following steps will be performed automatically and "Baseline" scenario will be registered.

Using `DataEngineer` class internally,

* Data downloading
* Data cleaning
* Data transformation
* Data subsetting
* Data complement (if necessary and selected)

Using `Dynamics` class internally,
* Time-series segmentation with S-R change point analysis
* Tau estimation
* ODE parameter estimation

In [ ]:
snr_act = cs.ODEScenario.auto_build(geo="Italy", model=cs.SIRFModel, complement=True)

We can perform simulation with `ODEScenario().simulate()` directly.

In [ ]:
snr_act.simulate(name="Baseline");

Or, we can confirm the dynamics of the baseline scenario, producing `Dynamics` instance.

In [ ]:
dyn_act = snr_act.to_dynamics(name="Baseline")
# Show summary
display(dyn_act.summary())
# Simulation
dyn_act_df = dyn_act.simulate(model_specific=False)
cs.line_plot(
    dyn_act_df.drop("Susceptible", axis=1), "Italy: Simulated data (Baseline scenario)")

If `name` is not specified, actual records will be shown.

In [ ]:
snr_act.simulate();

### 1.2 With `pandas.DataFrame`
The following solution is useful when we want to use `pandas.DataFrame` or sample data for scenario analysis.

At first, we will prepare an instance of `Dynamics`.

In [ ]:
dyn_sample = cs.Dynamics.from_sample(cs.SIRFModel, date_range=("01Jan2022", "31Jan2022"), tau=1440)
sample_df = dyn_sample.simulate()
cs.line_plot(sample_df.drop("Susceptible", axis=1), "Sample: Simulated data (Baseline scenario)")

Then, prepare a `pandas.DataFrame` with time-index and Population/Confirmed/Recovered/Fatal columns. `DataEngineer` class will be useful.

In [ ]:
eng = cs.DataEngineer(layers=["Country"])
df = sample_df.reset_index()
df.insert(0, "Country", "Sample")
eng.register(data=df)
eng.inverse_transform()
subset_df, *_ = eng.subset(
    geo="Sample", variables=["Population", "Confirmed", "Recovered", "Fatal"]
)
display(subset_df.head())
display(subset_df.tail())

Finally, we can create an instance of `ODEScenario`.

In [ ]:
snr = cs.ODEScenario(data=subset_df, location_name="Sample", complement=False)
snr.build_with_dynamics(name="Baseline", dynamics=dyn_sample)
# Show summary
snr.summary()

If we need to perform time-series segmentation, tau estimation and ODE parameter estimation, we can use `.build_with_model(name=<str>, model=<ODEModel>, data_range=<tuple of (str, str) or None>, tau=<int or None>` instead of `.build_with_dynamics()`.

## Set additional scenarios
We will define the following scenario as an example to analyze the impact of ODE parameter changes, copying the baseline scenario and add a phase with the changed ODE parameter values. Note that this is just an assumption. It will be better to change ODE parameter values for your analysis.

| name | 01Jan - 31Jan | $\kappa$ (to 30Jun) | $\rho$ (to 30Jun) | $\sigma$ (to 30Jun) |
|:---:|:---:|:---:|:---:|:---:|
| Baseline | 100% | 100% | 100% | 100% |
| Lockdown | 100% | 100% |  50% | 100% |
| Medicine | 100% |  50% | 100% | 200% |
| Vaccine  | 100% |  60% |  80% | 120% |

Get the baseline values of ODE parameters.

In [ ]:
df = snr.summary()
kappa, rho, sigma = df.loc[("Baseline", "0th"), ["kappa", "rho", "sigma"]]
print(f"kappa={kappa:.4}, rho={rho:.4}, sigma={sigma:.4}")

### 2.1 Lockdown scenario
This scenario assumes that $\rho$ value will be decreased from 01Feb2022 because of stay-home restriction.

In [ ]:
name = "Lockdown"
snr.build_with_template(name=name, template="Baseline")
snr.append(name=name, end="30Jun2022", rho=rho*0.5)
# Simulation
snr.simulate(name=name, v="01Feb2022");

### 2.2 Medicine scenario
This scenario assumes that $\kappa$ will be decreased and $\sigma$ will be increased because of new medicine development.

In [ ]:
name = "Medicine"
snr.build_with_template(name=name, template="Baseline")
snr.append(name=name, end="30Jun2022", kappa=kappa*0.5, sigma=sigma*2)
# Simulation
snr.simulate(name=name, v="01Feb2022");

### 2.3 Vaccine scenario
This scenario assumes that $\kappa$ and $\rho$ will be decreased and $\sigma$ will be increased because of vaccination.

In [ ]:
name = "Vaccine"
snr.build_with_template(name=name, template="Baseline")
snr.append(name=name, end="30Jun2022", kappa=kappa*0.6, rho=rho*0.8, sigma=sigma*1.2)
# Simulation
snr.simulate(name=name, v="01Feb2022");

### 2.4 Adjustment of the last dates
The last dates of the new scenarios are 30Jun2022, but the baseline scenario ends on 31Jan2022. We need to extend the last phase to the baseline scenario analysis with `ODEScenario().append(name="Baseline")` or `ODEScenario().append()` (useful when we have some unchanged scenarios).

Before appended:

In [ ]:
snr.summary().style.applymap(lambda x: "background-color: yellow", subset=pd.IndexSlice[("Baseline", "0th"), "End"])

Append:

In [ ]:
snr.append().summary().style.applymap(lambda x: "background-color: yellow", subset=pd.IndexSlice[("Baseline", "0th"), "End"])

If we need all values for dates, we can use `ODEScenario().track()` method.

In [ ]:
snr.track()

## 3. Compare and evaluate scenarios

### 3.1 Compare the params
We will confirm the ODE parameter values and reproduction number with `ODEScenario().compare_param(param)` method.

Reproduction number:

In [ ]:
snr.compare_param("Rt");

ODE parameter values:

In [ ]:
snr.compare_param("kappa");

In [ ]:
snr.compare_param("rho");

In [ ]:
snr.compare_param("sigma");

### 3.2 Compare simulated number of cases
We will compare simulated number of cases with `ODEScenario().compare_cases(variable, date_range=<None or tuple of dates>)` method.

In [ ]:
snr.compare_cases("Confirmed");

In [ ]:
snr.compare_cases("Fatal");

In [ ]:
snr.compare_cases("Recovered");

### 3.3 Describe representative values
We can use `ODEScenario.describe()` to describe representative values of simulated number of cases. Max value of Infected and the number of cases on the last date will be shown as a `pandas.DataFrame`. This is useful to confirm the impact of OE parameter changes.

In [ ]:
snr.describe()

### 3.4 Get representative scenario
Which scenario is the best/worst scenario? Specifying quantile, variable name and date, we can confirm that with `ODEScenario().represent(q=<float or tuple of float>, variable=<str>, date=<str or None>, included=<list of str or None>, excluded=<list of str or None>)`. `date=None` means the last date of scenarios.

In [ ]:
scenarios = snr.represent(q=(0.1, 0.9), variable="Confirmed", excluded=["Baseline"])
print(f" the best: {scenarios[0]} (small number of confirmed cases)\nthe worst: {scenarios[1]} (large number of confirmed cases)")

We can rename a scenario with `ODEScenario().rename(old, new)`.

In [ ]:
snr.rename(old=scenarios[0], new="Best")
snr.rename(old=scenarios[1], new="Worst").describe()

We can delete a scenario with `ODEScenario().delete(pattern, exact=True)`. Just for demonstration, we will delete the scenario which is not the baseline/best/worst scenario.

In [ ]:
deleted_scenarios = [name for name in snr.describe().index.unique() if name not in [*scenarios, "Best", "Worst", "Baseline"]]
deleted_scenarios

In [ ]:
for name in deleted_scenarios:
    snr.delete(name, exact=True)
snr.describe()

We can delete scenarios with regular expressions with `ODEScenario().delete(pattern, exact=True)`. Just for demonstration, we will add scenarios which names start with "Deleted_". Then, delete them.

In [ ]:
snr.build_with_template(name="Deleted_1", template="Baseline")
snr.build_with_template(name="Deleted_2", template="Baseline").describe()

In [ ]:
snr.delete("Deleted_", exact=False).describe()

As explained, we can compare the scenarios with the methods, `ODEScenario().compare_cases()` as an example.

In [ ]:
snr.compare_cases("Confirmed");

Thank you!